# Import modules

In [ ]:
import os # to get current working directory
import numpy as np # to read example data
import trspecfit as trsfit # main module

# Example: Two peaks with dependent parameters (+offset +background)

## Create a project (defines axis labels among other parameters)

In [ ]:
project_folder = os.getcwd() # path to data folder(s)
P = trsfit.Project(path=project_folder)

## Create a file (the fit models will be associated with this)

In [ ]:
file_folder = "data" # define where data of our file is located (in project folder)
# load data into a trsfit.File
F = trsfit.File(P, 
                path = file_folder,
                data = np.loadtxt(P.path / file_folder / "data.csv", delimiter=','),
                energy = np.loadtxt(P.path / file_folder / "energy.csv"),
                time = np.loadtxt(P.path / file_folder / "time.csv")
               )

## Set limits (in time and energy) for the fit models

In [ ]:
F.set_fit_limits(energy_limits = [91, 81.5],
                 time_limits = [-1.5, 10])

## Define the baseline/ ground state/ unbiased portion of the spectrum 

In [ ]:
F.define_baseline(-1.5, 0, time_type='abs') # use absolute time values ...
#F.define_baseline(0, 19, time_type='ind') # ... or start and stop index

In [ ]:
F.reset_models() # use this to delete all models previously loaded in this file

## Load a model for the basline spectrum and fit it

In [ ]:
model_name_base = "base" # select model name

F.load_model(model_yaml = "models_Au4f.yaml",
             model_info = [model_name_base,])
F.describe_model(model_info=model_name_base, detail=0)

In [ ]:
F.fit_baseline(model_name = model_name_base, fit = 2)

## Load a model to fit data "Slice-by-Slice"

In [ ]:
model_name_SbS = "2D" # select model name

F.load_model(model_yaml = "models_Au4f.yaml",
             model_info = [model_name_SbS,])
F.describe_model(model_name_SbS, detail=0)

In [ ]:
F.fit_SliceBySlice(model_name = model_name_SbS,
                   fit = 1,
                   try_CI = 0)

## Load a model to perform a global 2D fit

In [ ]:
# load energy/spectroscopy model
model_name_2D = "2D" # select model name
F.delete_model(model_name_2D)
F.load_model(model_yaml = "models_Au4f.yaml",
             model_info = [model_name_2D,])
F.describe_model(model_info=model_name_2D, detail=0)

In [ ]:
# add time dependence for one or more parameters
F.add_time_dependence(model_yaml = "models_time.yaml",
                      model_info = ["IRF", "MonoExpNeg", "MonoExpPos"],
                      par_name = "GLP_01_x0",
                      frequency = 0.5)
print()
F.describe_model(model_info=[model_name_2D,], detail=1)

In [ ]:
# perform fit
F.fit_2Dmodel(model_name = model_name_2D,
              fit = 2,
              try_CI = 0)